# Data Processing for CS1710

In [64]:
# Imports
import pandas as pd
import numpy as np
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler
from matplotlib.lines import Line2D


In [65]:
# Display options
pd.set_option('display.max_rows', 100)

In [66]:
# Read CSV data
voter_blocks_all = pd.read_csv('../data/MA_l2_2022stats_2020block/MA_l2_2022stats_2020block.csv', low_memory=False).set_index('geoid20')
display(voter_blocks_all.shape)

(107292, 395)

## Data Cleaning and Processing

### Define and Rename Columns

We define the columns of interest and rename the columns for language and gender for easier interpretation.

In [67]:
# Group demographic columns together
registered = ['total_reg']
age = ['age_18_19', 'age_20_24', 'age_25_29','age_30_34','age_35_44', 'age_45_54', 'age_55_64', 'age_65_74','age_75_84', 'age_85over']
old_gender = [ 'voters_gender_m', 'voters_gender_f', 'voters_gender_unknown'] 
gender = [ 'gender_m', 'gender_f', 'gender_unknown'] 
party = ['party_npp', 'party_dem', 'party_rep','party_lib', 'party_grn', 'party_con', 'party_ain', 'party_scl','party_oth', 'party_unk']
ethnicity1 = ['eth1_eur', 'eth1_hisp', 'eth1_aa',
                'eth1_esa', 'eth1_oth', 'eth1_unk']
ethnicity2 = ['eth2_euro', 'eth2_64',
                'eth2_93', 'eth2_10', 'eth2_30', 'eth2_23', 'eth2_66', 'eth2_34',
                'eth2_21', 'eth2_35', 'eth2_14', 'eth2_12', 'eth2_55', 'eth2_13',
                'eth2_32', 'eth2_61', 'eth2_85', 'eth2_29', 'eth2_33', 'eth2_38',
                'eth2_15', 'eth2_57', 'eth2_19', 'eth2_26', 'eth2_59', 'eth2_unk']
old_languages = ['languages_description_english', 'languages_description_spanish',
                'languages_description_portuguese',
                'languages_description_chinese', 'languages_description_italian',
                'languages_description_vietnamese', 'languages_description_other',
                'languages_description_unknown']
languages = ['lang_english', 'lang_spanish',
                'lang_portuguese',
                'lang_chinese', 'lang_italian',
                'lang_vietnamese', 'lang_other',
                'lang_unknown']
income = ['commercialdata_estimatedhhincomeamount_avg']

# Group variables related to the 2020 presidential election
presidential_2020 = ['g20201103_voted_all', 'g20201103_reg_all', 'g20201103_pct_voted_all',
                    'g20201103_voted_gender_m', 'g20201103_reg_gender_m',
                    'g20201103_pct_voted_gender_m', 'g20201103_voted_gender_f',
                    'g20201103_reg_gender_f', 'g20201103_pct_voted_gender_f',
                    'g20201103_voted_gender_unk', 'g20201103_reg_gender_unk',
                    'g20201103_pct_voted_gender_unk', 'g20201103_voted_eur',
                    'g20201103_reg_eur', 'g20201103_pct_voted_eur', 'g20201103_voted_hisp',
                    'g20201103_reg_hisp', 'g20201103_pct_voted_hisp', 'g20201103_voted_aa',
                    'g20201103_reg_aa', 'g20201103_pct_voted_aa', 'g20201103_voted_esa',
                    'g20201103_reg_esa', 'g20201103_pct_voted_esa', 'g20201103_voted_oth',
                    'g20201103_reg_oth', 'g20201103_pct_voted_oth', 'g20201103_voted_unk',
                    'g20201103_reg_unk', 'g20201103_pct_voted_unk']

# Define columns with counts for voters registered and voted in 2020 presidential election (excludes percentages)
presidential_counts = [col for col in presidential_2020 if not '_pct_' in col]

In [68]:
# Rename columns for easier intepretation
col_labels = {k:v for k, v in zip(old_gender, gender)}
col_labels.update({k:v for k,v in zip(old_languages, languages)})
voter_blocks_all = voter_blocks_all.rename(col_labels, axis=1)

### Missing Data Handling

#### 'NO BLOCK ASSIGNMENT' entries

We are choosing to ignore and drop these values because of inappropriate assumptions that need to be made if trying to disaggregate these entries.

Similarly, using counties as our unit of observation is difficult due to there only being 14 counties.

It seems as though counties with lower population have higher proportion of unassigned voters.

We will also explore the excluded data to determine which population groups we might be disproportionately excluding.

In [69]:
# Drop 'NO BLOCK ASSIGNMENT' entries
voter_blocks = voter_blocks_all[~voter_blocks_all.index.str.contains('NO BLOCK ASSIGNMENT')].copy()

#### Null voter turnout percentages (DONT DO THIS FOR CS1710?)

Some Census Block Groups have missing percentage voter turnout for the 2020 presidential election due to not having any voters registered for that election.

We choose to drop Census Block Groups that have zero voters registered for the 2020 presidential election (and total voters registered), as they are useless for our analysis.

In [70]:
# Drop Census Blocks with zero voters registered during the 2020 presidential election or ever
# voter_blocks = voter_blocks[voter_blocks['g20201103_reg_all'] > 0]
# voter_blocks = voter_blocks[voter_blocks['total_reg'] > 0]

###  Aggregation to Block Groups

Some Census Blocks are incredibly small, causing proportions to change wildly for small changes in counts.

We will aggregate the data to Census Block Groups to minimize the effects of incredibly small samples in Census Blocks.

#### Impute income data 

We are aggregating our data to the Census Block Group level and taking the mean of the mean household income at the Census Block level weighted by the number of total voters in each Census Block.

Even after imputation, there are 4 Census Block Groups still missing income data, with 14 voters in those Census Block Groups (0.00031% of the total).

We chose to drop these observations

In [71]:
# Weighted mean function based on total registered voters
wm = lambda x: (
    np.average(x.dropna(), weights=voter_blocks.loc[x.dropna().index, 'total_reg'])
    if voter_blocks.loc[x.dropna().index, 'total_reg'].sum() > 0
    else np.nan
)

# Define aggregation method for columns
agg_funcs = {col: 'sum' for col in [*registered, *age, *gender, *party, *ethnicity1, *languages, *presidential_counts]}
agg_funcs.update({'commercialdata_estimatedhhincomeamount_avg': wm})

# Define block group ID
voter_blocks['block_group_id'] = voter_blocks.index.str[:12]
block_groups = voter_blocks.groupby('block_group_id').agg(agg_funcs)

# Rename the income column
block_groups = block_groups.rename({'commercialdata_estimatedhhincomeamount_avg':'mean_hh_income'}, axis=1)

In [72]:
block_groups[block_groups['mean_hh_income'].isna()]

,total_reg,age_18_19,age_20_24,age_25_29,age_30_34,age_35_44,age_45_54,age_55_64,age_65_74,age_75_84,...,g20201103_reg_hisp,g20201103_voted_aa,g20201103_reg_aa,g20201103_voted_esa,g20201103_reg_esa,g20201103_voted_oth,g20201103_reg_oth,g20201103_voted_unk,g20201103_reg_unk,mean_hh_income
block_group_id,,,,,,,,,,,,,,,,,,,,,
250010102082,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
250056171013,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
250059900000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
250079900000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
250092032005,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
250099901000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
250158219032,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
250173537003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
250173593011,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN


In [73]:
print(f'After imputation, there are a total of {len(block_groups[block_groups['mean_hh_income'].isna()].index)} Census Block Groups still missing income data, with {block_groups[block_groups['mean_hh_income'].isna()]['total_reg'].sum()} voters in those Census Block Groups ({block_groups[block_groups['mean_hh_income'].isna()]['total_reg'].sum()/block_groups['total_reg'].sum():.5%} of the total).')

After imputation, there are a total of 38 Census Block Groups still missing income data, with 14 voters in those Census Block Groups (0.00031% of the total).


In [74]:
# Choose to drop NaN values for income due to low number of voters in these blocks.
block_groups = block_groups.dropna(subset='mean_hh_income')

#### Only keep columns of interest

We drop all other columns that are not of interest. We discovered that voter counts with 'unknown' party are extremely few (at most 1 voter per Census Block) and that after dropping Census Blocks with zero voters registered for the 2020 presidential election, all values for this column are 0. So, we can drop this column without losing any information.

In [75]:
# Only keep columns of interest
party = ['party_npp', 'party_dem', 'party_rep','party_lib', 'party_grn', 'party_con', 'party_ain', 'party_scl','party_oth']
predictors = [*registered, *age, *gender, *party, *ethnicity1, *languages, 'mean_hh_income']
# Other potential response variables could be g20201103_voted_all or those who voted and are a part of a certain demographic group
response = 'g20201103_pct_voted_all'
block_groups[response] = block_groups['g20201103_voted_all'] / block_groups['g20201103_reg_all']

### Aggregate to Tract and County level

In [76]:
# Weighted mean function based on total registered voters
wm = lambda x: (
    np.average(x.dropna(), weights=block_groups.loc[x.dropna().index, 'total_reg'])
    if block_groups.loc[x.dropna().index, 'total_reg'].sum() > 0
    else np.nan
)

# Define aggregation method for columns
agg_funcs = {col: 'sum' for col in [*registered, *age, *gender, *party, *ethnicity1, *languages, *presidential_counts]}
agg_funcs.update({'mean_hh_income': wm})

# Define tract and county IDs
block_groups['tract_id'] = block_groups.index.str[:10]
block_groups['county_id'] = block_groups.index.str[:5]

# Aggregate
tracts = block_groups.groupby('tract_id').agg(agg_funcs)
counties = block_groups.groupby('county_id').agg(agg_funcs)


### Take Proportions

Instead of using demographic counts as our predictors, we are choosing to use demographic proportions (i.e. the percentage of people that identify as _ in each Census Block Group).

We will use the number of voters registered in each Census Block Group as weights in our analysis to account for the fact that Census Block Groups with low voter registration counts can have wildly different demographic proportions.

In [77]:
# Take proportions
for cat in [*age, *gender, *party, *ethnicity1, *languages]:
    block_groups[cat + '_pct'] = block_groups[cat] / block_groups['total_reg']
    tracts[cat + '_pct'] = tracts[cat] / tracts['total_reg']
    counties[cat + '_pct'] = counties[cat] / counties['total_reg']

### Join to Shapefile

Finally, we join the data to Census Block Groups shapefile for visualization purposes and to potentially use the geographic locations of Census Blocks in our analysis.

In [78]:
# Read shapefiles
block_groups_shp = gpd.read_file('../data/ma_pl2020_bg/ma_pl2020_bg.shp')
display(block_groups_shp.shape)
tracts_shp = gpd.read_file('../data/ma_pl2020_t/ma_pl2020_t.shp')
display(tracts_shp.shape)
counties_shp = gpd.read_file('../data/ma_pl2020_cnty/ma_pl2020_cnty.shp')
display(counties_shp.shape)

(5116, 346)

(1620, 345)

(14, 349)

In [79]:
bg_gdf = block_groups_shp.merge(block_groups, left_on='GEOID20', right_on='block_group_id').set_index('GEOID20').rename({'P0010001':'total_pop','P0030001':'18+_pop'}, axis=1)
t_gdf = block_groups_shp.merge(tracts, left_on='GEOID20', right_on='tract_id').set_index('GEOID20').rename({'P0010001':'total_pop','P0030001':'18+_pop'}, axis=1)
c_gdf = block_groups_shp.merge(counties, left_on='GEOID20', right_on='county_id').set_index('GEOID20').rename({'P0010001':'total_pop','P0030001':'18+_pop'}, axis=1)

In [80]:
# Finalize columns

# keep_cols = predictors + [response] + presidential_counts + ['tract_id','county_id','ALAND20', 'AWATER20', 'geometry', 'total_pop', '18+_pop', 'total_nh_white', '18+_nh_white', 'total_hisp', '18+_hisp', 'total_ap_black', '18+_ap_black', 'total_ap_asian', '18+_ap_asian']
# keep_cols = predictors + [response] + presidential_counts + ['tract_id','county_id','ALAND20', 'AWATER20', 'geometry']
keep_cols = predictors + ['g20201103_voted_all','g20201103_reg_all','BASENAME', 'ALAND20', 'AWATER20', 'geometry']

bg_gdf = bg_gdf[keep_cols]
t_gdf = t_gdf[keep_cols]
c_gdf = c_gdf[keep_cols]

for gdf in [bg_gdf,t_gdf,c_gdf]:
    gdf['2020_turnout_pct'] = gdf['g20201103_voted_all'] / gdf['g20201103_reg_all']
    gdf['2020_absent_pct'] = 1 - gdf['2020_turnout_pct']

### Final Data

After all processing has been done:
The number of voters registered has dropped from **4,606,321** to **4,532,271**.
**1.61%** of total registered voters were dropped.

### Export Data

In [81]:
# Optionally export the data for CS1710
bg_gdf.to_file('../data/block_groups.geojson', driver='GeoJSON')
t_gdf.to_file('../data/tracts.geojson', driver='GeoJSON')
c_gdf.to_file('../data/counties.geojson', driver='GeoJSON')